---
# <center>Sberbank Russian Housing Market</center>

#### Data Files
---
* train.csv, test.csv: информация об отдельных сделках. Строки индексируются по полю «id», которое относится к отдельным транзакциям (определенные свойства могут появляться более одного раза в отдельных транзакциях). Эти файлы также содержат дополнительную информацию о районе каждого объекта недвижимости.

* macro.csv: данные по макроэкономике и финансовому сектору России (могут быть присоединены к набору поездов и тестов в столбце «timestamp»)

* sample_submission.csv: пример сабмита
* data_dictionary.txt: объяснения полей, доступных в других файлах данных

## Импорт модулей
---

In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import shap
import lightgbm as lgb
import seaborn as sns
import xgboost as xgb
import zipfile
from sklearn import model_selection, preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor  
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, r2_score
import catboost as cb
import warnings

color = sns.color_palette()
%matplotlib inline
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)

## Exploratory Data Analysis
---

In [ ]:
train_zip = zipfile.ZipFile('/kaggle/input/sberbank-russian-housing-market/train.csv.zip') 
test_zip = zipfile.ZipFile('/kaggle/input/sberbank-russian-housing-market/test.csv.zip')
sample_submit_zip = zipfile.ZipFile('/kaggle/input/sberbank-russian-housing-market/sample_submission.csv.zip') 
macro_zip = zipfile.ZipFile('/kaggle/input/sberbank-russian-housing-market/macro.csv.zip') 


df_train = pd.read_csv(train_zip.open('train.csv'))
df_test = pd.read_csv(test_zip.open('test.csv'))
sample_submit = pd.read_csv(sample_submit_zip.open('sample_submission.csv'))
df_macro = pd.read_csv(macro_zip.open('macro.csv'))

print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)

Для начала рассмотрим датасет в целом, и наш таргет

In [ ]:
df_train.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.distplot(df_train['price_doc'].values, bins=100, kde=True, ax=ax)
plt.xlabel('price_doc', fontsize=12)
plt.show()

Невооруженным взглядом видно ненормальное распределение, стоит применить логарифмирование

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.distplot(np.log(df_train['price_doc']).values, bins=100, kde=True, ax=ax)
plt.xlabel('log(price_doc)', fontsize=12)
plt.show()

Логарифмирование нашего таргета действительно помогло, данные стало удобней воспринимать. Заменим этот признак в датасете

In [ ]:
df_train['price_doc_log'] = np.log(df_train['price_doc'])

Рассмотрим корреляции нашего целевого признака с другими:

In [ ]:
corrs = df_train.corr(method='pearson')
corr_with_prices = corrs["price_doc"][:-1]
corr_with_prices[abs(corr_with_prices).argsort()[::-1]].tail(100)

Как оказалось, как минимум целая сотня признаков практически никаким образом не коррелирует с нашим таргетом. Есть вариант удалить их.

In [ ]:
toDelete = list(corr_with_prices[abs(corr_with_prices).argsort()[::-1]].tail(100).index)
print(toDelete[0], toDelete[-1]) # Убеждаемся, что взяли те признаки

In [ ]:
print(df_train.shape, '\n', df_test.shape)

for col in toDelete:
    df_train.drop(col, axis=1, inplace=True)
    df_test.drop(col, axis=1, inplace=True)

print(df_train.shape, '\n', df_test.shape)

In [ ]:
temp_df = df_train.groupby(['floor'])['price_doc'].aggregate(np.median).reset_index()
plt.figure(figsize=(10, 8))
sns.pointplot(x='floor', y='price_doc', data=temp_df)
plt.ylabel('Средняя цена', fontsize=12)
plt.xlabel('Номер этажа', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

Видим странную аномалию цены для 33-этажного дома, удалим её.

In [ ]:
df_train[(df_train['floor']) == 33]

In [ ]:
df_train.drop(df_train.index[7457], inplace=True)

## Feature Engineering
---

Воспользуемся признаком timestamp чтобы извлечь из него два новых признака - год и месяц

In [ ]:
df_train['timestamp'][0:3]

In [ ]:
df_train['year'] = df_train['timestamp'].apply(lambda x: x[:4]).astype(int)
df_train['month'] = df_train['timestamp'].apply(lambda x: x[5:7]).astype(int)

df_test['year'] = df_test['timestamp'].apply(lambda x: x[:4]).astype(int)
df_test['month'] = df_test['timestamp'].apply(lambda x: x[5:7]).astype(int)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
sns.barplot(x='month', y='price_doc', data=df_train)
plt.title('Цена продажи по месяцам', fontsize=18)
plt.xticks(rotation='vertical')
plt.show()

Проверим наш датасет на пропуски

In [ ]:
missingValues = df_train.columns[df_train.isnull().any()].tolist()

pd.isnull(df_train[missingValues]).sum().sort_values(ascending=False)

Пропусков оказалось очень много. Заменим часть из них модой, а часть средним значением

In [ ]:
cols_fillna_mode = ['floor',
 'product_type',
 'num_room',
 'state',
 'hospital_beds_raion',
 'build_count_brick',
 'build_count_monolith',
 'green_part_2000']

cols_fillna_mean = ['life_sq',
 'metro_min_walk',
 'metro_km_walk',
 'railroad_station_walk_km',
 'railroad_station_walk_min',
 'cafe_sum_1500_min_price_avg',
 'cafe_sum_1500_max_price_avg',
 'cafe_avg_price_1500',
 'cafe_sum_2000_max_price_avg',
 'cafe_avg_price_2000']

In [ ]:
for col in cols_fillna_mode:
    df_train[col].fillna(df_train[col].mode().iloc[0],inplace=True)
    df_test[col].fillna(df_train[col].mode().iloc[0],inplace=True)

for col in cols_fillna_mean:
    df_train[col].fillna(df_train[col].mean(),inplace=True)
    df_test[col].fillna(df_train[col].mean(),inplace=True)

In [ ]:
numerical_features = df_train.dtypes[df_train.dtypes != "object"].index
categorical_features = df_train.dtypes[df_train.dtypes == "object"].index

print("Кол-во количественных признаков: ", len(numerical_features))
print("Кол-во категориальных признаков: ", len(categorical_features))

In [ ]:
df_train.isna().sum().sort_values(ascending=False)

In [ ]:
df_train.drop(['id', 'price_doc', 'timestamp'], axis=1, inplace=True)
id_test = df_test['id']
df_test.drop(['id', 'timestamp'], axis=1, inplace=True)

In [ ]:
numerical_features = df_train.dtypes[df_train.dtypes != "object"].index
categorical_features = df_train.dtypes[df_train.dtypes == "object"].index

print("Кол-во количественных признаков: ", len(numerical_features))
print("Кол-во категориальных признаков: ", len(categorical_features))

Используем One Hot Encoder и закодируем наши категориальные переменные, чтобы нашим моделькам было удобнее учиться.

In [ ]:
encoder = OneHotEncoder(handle_unknown='error')
encoder_cols_train = pd.DataFrame(encoder.fit_transform(df_train[categorical_features]).toarray())
encoder_cols_test = pd.DataFrame(encoder.transform(df_test[categorical_features]).toarray())

In [ ]:
# вернем названия и индексы
encoder_cols_train.columns = encoder.get_feature_names(categorical_features)
encoder_cols_test.columns = encoder.get_feature_names(categorical_features)

encoder_cols_train.index = df_train.index
encoder_cols_test.index = df_test.index

In [ ]:
num_df_train = df_train.drop(categorical_features, axis=1)
num_df_test = df_test.drop(categorical_features, axis=1)

In [ ]:
df_train_encoded = pd.concat([num_df_train, encoder_cols_train], axis=1)
df_test_encoded = pd.concat([num_df_test, encoder_cols_test], axis=1)

print("Train dataset shape:", df_train_encoded.shape)
print("Test dataset shape:", df_test_encoded.shape)

In [ ]:
df_train_encoded.median().sort_values(ascending=False)

In [ ]:
X = df_train_encoded.drop(['price_doc_log'], axis=1)
y = df_train_encoded['price_doc_log']

print("X shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2022)

X_test = df_test_encoded

## Model Building
---

#### Decision Tree

Для начала попробуем обучить обучное дерево решений и посмотрим на метрику

In [ ]:
tree = DecisionTreeRegressor(random_state=2022, max_depth=5, min_samples_split=20)  

tree.fit(X_train, y_train)
tree_predictions_log = tree.predict(X_val)
tree_predictions = np.exp(tree_predictions_log)

In [ ]:
print('RMSLE:', np.sqrt(mean_squared_log_error(np.exp(y_val), tree_predictions)))

In [ ]:
predict = np.exp(tree.predict(X_test))
submission = pd.DataFrame({'id': id_test, 'price_doc': predict})
submission.head()

In [ ]:
submission.to_csv('DecisionTree.csv', index=False)

Метрика не очень впечатляющая, так что попробуем использовать бустинги

#### XGBoost

In [ ]:
dmatrix_train = xgb.DMatrix(X_train, y_train) # _scaled
dmatrix_val = xgb.DMatrix(X_val, y_val)

dmatrix_test = xgb.DMatrix(X_test)

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 1.0,
    'colsample_bytree': 0.7,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'verbosity': 0
}

partial_model = xgb.train(xgb_params, dmatrix_train, num_boost_round=1000, evals=[(dmatrix_val, 'val')],
                       early_stopping_rounds=20, verbose_eval=20)

num_boost_round = partial_model.best_iteration

In [ ]:
model = xgb.train(dict(xgb_params, verbose=1), dmatrix_train, num_boost_round=num_boost_round)

In [ ]:
predict = np.exp(model.predict(dmatrix_val))
print('RMSLE:', np.sqrt(mean_squared_log_error(np.exp(y_val), predict)))

Метрика стала несколько лучше. Посмотрим на анализ важности признаков от библиотеки SHAP

In [ ]:
ylog_pred = model.predict(dmatrix_test)
y_pred = np.exp(ylog_pred)

submission = pd.DataFrame({'id': id_test, 'price_doc': y_pred})
submission.head()

In [ ]:
submission.to_csv("XGB_new_clear_submission.csv", index=False)

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(dmatrix_test)
shap.summary_plot(shap_values, X_test)

Сразу же видно, что самый большой ипакт вносит полная площадь недвижимости, что неудивительно. Но также можно заметить, что кафе тоже повлиял на модель

#### XGBoost (+ Cross Validation)

In [ ]:
dmatrix_train = xgb.DMatrix(X_train, y_train)
dmatrix_test = xgb.DMatrix(X_test)

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'verbosity': 0
}

In [ ]:
cv_output = xgb.cv(xgb_params, 
                   dmatrix_train, 
                   num_boost_round=1000, 
                   early_stopping_rounds=20,
                   verbose_eval=50, 
                   show_stdv=False)

num_boost_rounds = len(cv_output)

In [ ]:
model = xgb.train(dict(xgb_params, verbose=1), dmatrix_train, num_boost_round=num_boost_rounds)

In [ ]:
predict = np.exp(model.predict(dmatrix_test))
submission = pd.DataFrame({'id': id_test, 'price_doc': predict})
submission.head()

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(dmatrix_test)
shap.summary_plot(shap_values, X_test)

In [ ]:
submission.to_csv('XGB_CV.csv', index=False)

#### PCA + XGBoost

Попробуем применить метод снижения размерности, и обучить бустинг уже на нем

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2022)

X_test = df_test_encoded

In [ ]:
pca = PCA(n_components=20).fit(X_train)

X_train_pca=pca.transform(X_train)
X_val_pca=pca.transform(X_val)

In [ ]:
dmatrix_train = xgb.DMatrix(X_train_pca, y_train)
dmatrix_val = xgb.DMatrix(X_val_pca, y_val)
dmatrix_test = xgb.DMatrix(X_test)

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 1.0,
    'colsample_bytree': 0.7,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'verbosity': 0
}

partial_model = xgb.train(xgb_params, 
                          dmatrix_train,
                          num_boost_round=1000, 
                          evals=[(dmatrix_val, 'val')],
                          early_stopping_rounds=20, 
                          verbose_eval=20)

num_boost_round = partial_model.best_iteration

In [ ]:
model = xgb.train(dict(xgb_params, verbose=1), dmatrix_train, num_boost_round=num_boost_round)

In [ ]:
predict = np.exp(model.predict(dmatrix_val))
rmsle = np.sqrt(mean_squared_log_error(np.exp(y_val), predict))

print('RMSLE: {:.3f}'.format(rmsle))

Скор заметно упал. Возможно, метод главных компонент не подойдет для этой задачи

In [ ]:
# predict = np.exp(model.predict(dmatrix_test))
# submission = pd.DataFrame({'id': id_test, 'price_doc': predict})
# submission.head()

In [ ]:
# submission.to_csv('XGB_PCA.csv', index=False)

В этот раз взглянем на график важности фичей, который предоставляет XGBoost

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 20))
xgb.plot_importance(model, max_num_features=20, height=0.5, ax=ax);

Похоже, что каждая из 20 главных компонент оказалась важной для модели

#### CatBoost

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2022)

X_test = df_test_encoded

In [ ]:
train_dataset = cb.Pool(X_train, y_train) 
test_dataset = cb.Pool(X_val, y_val)

In [ ]:
model = cb.CatBoostRegressor(loss_function='RMSE')

In [ ]:
grid = {'iterations': [150, 200],
        'learning_rate': [0.03, 0.05],
        'depth': [5, 7],
        'l2_leaf_reg': [1]}

model.grid_search(grid, train_dataset)

In [ ]:
pred = np.exp(model.predict(X_val))
rmsle = np.sqrt(mean_squared_log_error(np.exp(y_val), pred))

print('RMSLE: {:.3f}'.format(rmsle))

In [ ]:
predict = np.exp(model.predict(X_test))
submission = pd.DataFrame({'id': id_test, 'price_doc': predict})
submission.head()

In [ ]:
submission.to_csv('Catboost.csv', index=False)